In [1]:
!pip install streamlit pymupdf pytesseract Pillow nltk spacy google-generativeai pyngrok
!sudo apt-get install tesseract-ocr
!python -m spacy download en_core_web_sm

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
with open('InvoiceExtracter.py', 'w') as f:
    f.write("""
import streamlit as st
import tempfile
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import os
import nltk
import google.generativeai as genai
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Initialize Streamlit app
st.title('Invoice Details Extractor 🧾')
st.markdown('<style>h1{color: orange; text-align: center;}</style>', unsafe_allow_html=True)
st.subheader('Extract Customer Details, Products, and Total Amount from Invoice PDFs')

# File upload widget
uploaded_files = st.sidebar.file_uploader("Upload PDF files", type=["pdf"], accept_multiple_files=True)

# Configure the LLM API with your API key
api_key = "YOUR_API_KEY"
genai.configure(api_key=api_key)

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

def extract_text_and_images(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text_content = ""

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text_content += page.get_text()

        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image = Image.open(io.BytesIO(image_bytes))

            # Save image temporarily
            temp_image_path = f"temp_image.{image_ext}"
            image.save(temp_image_path)

            # Extract text from image using Tesseract
            text_content += pytesseract.image_to_string(Image.open(temp_image_path))

            # Clean up temporary image file
            os.remove(temp_image_path)

    return text_content

def extract_invoice_details(text):
    model = genai.GenerativeModel('gemini-1.5-flash')

    prompt = f"Extract customer details, products, and total amount from the following invoice text: {text}"

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        st.error(f"Failed to extract invoice details: {e}")
        return None

def display_invoice_details(details):
    st.subheader("Extracted Invoice Details")
    st.write(details)

if uploaded_files:
    all_texts = ""
    for uploaded_file in uploaded_files:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
            temp_file.write(uploaded_file.read())
            file_path = temp_file.name
            all_texts += extract_text_and_images(file_path) + " "
            os.remove(file_path)

    invoice_details = extract_invoice_details(all_texts)
    if invoice_details:
        display_invoice_details(invoice_details)
    """)

In [5]:
from pyngrok import ngrok
ngrok.set_auth_token("YOUR_AUTH_TOKEN")

# Open a tunnel on port 8501 for streamlit
public_url = ngrok.connect(addr='8501', proto='http')
print(f'Public URL: {public_url}')

# Run the Streamlit app
!streamlit run InvoiceExtracter.py --server.port 8501

Public URL: NgrokTunnel: "https://e723-35-199-32-199.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.199.32.199:8501

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt 